In [1]:
!pip install python-wordpress-xmlrpc

In [6]:
!pip install selenium
!pip install gspread

In [2]:
!pip install oauth2client

In [2]:
#①
import os
from selenium import webdriver
import time
import pandas as pd
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support.ui import Select
import urllib
from bs4 import BeautifulSoup
import re
import csv
import codecs
import requests
import urllib.request as req
import openpyxl
import glob
from sklearn.datasets import  load_iris
import json
import time
import datetime
from datetime import datetime, date, timedelta
import numpy as np
from matplotlib.ticker import MaxNLocator
import matplotlib.ticker as ticker
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Hiragino Maru Gothic Pro', 'Yu Gothic', 'Meirio', 'Takao', 'IPAexGothic', 'IPAPGothic', 'VL PGothic', 'Noto Sans CJK JP']
import matplotlib.pyplot as plt
import pandas as pd
import pandas as pd
from dateutil.relativedelta import relativedelta
from PIL import Image, ImageDraw, ImageFont
from wordpress_xmlrpc import Client, WordPressPost
from wordpress_xmlrpc.compat import xmlrpc_client
from wordpress_xmlrpc.methods import media, posts
import time
from wordpress_xmlrpc import Client, WordPressPost
from wordpress_xmlrpc.methods.users import GetUserInfo
from wordpress_xmlrpc.methods.posts import GetPosts, NewPost
import gspread
from oauth2client.service_account import ServiceAccountCredentials

print('ライブラリの読み込み完了')
def set_round(x):
    x = (round(x, 1))
    return x

def set_round_int(x):
    x = int(round(x, 0))
    return x

ライブラリの読み込み完了


In [12]:
#/Users/macbook/Downloads/20210322_ワンダーランド香椎Ⅱ.xlsx
pd.set_option('display.max_rows', 8)
#pd.set_option('display.max_rows', None)


#ここは記入する
title_text_only = "20210713_ビーム原"
filename= r"C:\Users\81807\Documents\元データ　原.xlsx"

sheet_df = pd.read_excel(filename, usecols=[0, 1, 2,9])
display(sheet_df)
sheet_df.columns = ['台番', '機種', 'G数','差枚']
sheet_df['G数'] = sheet_df['G数'].astype(int)
sheet_df['差枚'] = sheet_df['差枚'].astype(int)
sheet_df['出率'] = (sheet_df['G数'] * 3 + sheet_df['差枚'])
sheet_df['出率'] = sheet_df['出率'] / (sheet_df['G数'] * 3) * 100
sheet_df = sheet_df.fillna(0)
sheet_df['出率'] =sheet_df['出率'].map(set_round)
sheet_df = sheet_df.reindex(columns=['機種', '台番', '差枚','G数','出率'])
df_a = ichiran_df = sheet_df
display(df_a)

kisyubetu_sheet_df = sheet_df.groupby(['機種']).mean()
kisyubetu_sheet_df['差枚'] = kisyubetu_sheet_df['差枚'].map(set_round_int)
kisyubetu_sheet_df['出率'] = kisyubetu_sheet_df['出率'].map(set_round)
kisyubetu_sheet_df['G数'] = kisyubetu_sheet_df['G数'].map(set_round_int)
kisyu_matome_df = kisyubetu_sheet_df = kisyubetu_sheet_df.drop('台番', axis=1)
kisyu_matome_df.reset_index(inplace=True, drop=False)
kisyubetu_sheet_df['台数'] = list(ichiran_df.groupby('機種').count()['台番'])
bariety_df = kisyubetu_sheet_df[kisyubetu_sheet_df['台数'] == 1]
kisyubetu_sheet_df = kisyubetu_sheet_df[kisyubetu_sheet_df['台数'] != 1]

kisyu_name_list = []

for kisyu_name in list(kisyubetu_sheet_df['機種']):
    #print(kisyu_name)
    df_b = ichiran_df.query('機種.str.contains(@kisyu_name)', engine='python')
    #display(df_b)
    win_rate_bunshi = 0
    for x in list(df_b['差枚']):
        if x > 0:
            win_rate_bunshi += 1
        else:
            pass
    kisyu_name_list.append(win_rate_bunshi)

kisyubetu_sheet_df['勝利台数'] = kisyu_name_list
kisyubetu_sheet_df['勝利台数'] = kisyubetu_sheet_df['勝利台数'].astype(str)
kisyubetu_sheet_df['台数'] = kisyubetu_sheet_df['台数'].astype(str)

kisyubetu_sheet_df['勝率'] = kisyubetu_sheet_df['勝利台数'] + '/' + kisyubetu_sheet_df['台数']
kisyubetu_sheet_df = kisyubetu_sheet_df.drop(['勝利台数','台数'], axis=1)
kisyubetu_sheet_df = kisyubetu_sheet_df.reindex(columns=['機種', '差枚','G数','勝率','出率'])
kisyubetu_sheet_df = kisyubetu_sheet_df.sort_values('差枚', ascending=False)
display(kisyubetu_sheet_df)


,台番,機種,G数,差枚
0,1,ＳＬＯＴ魁！！男塾,430,-290
1,2,燃えよ！功夫淑女ドラゴン,4070,-70
2,3,パチスロ ゼクスイグニッション,1990,-1670
3,5,麻雀格闘倶楽部２,2370,-320
...,...,...,...,...
307,707,プレミアムハナハナ-30,7380,2810
308,708,プレミアムハナハナ-30,7220,-70
309,710,プレミアムハナハナ-30,9010,4160
310,711,プレミアムハナハナ-30,8850,4340


,機種,台番,差枚,G数,出率
0,ＳＬＯＴ魁！！男塾,1,-290,430,77.5
1,燃えよ！功夫淑女ドラゴン,2,-70,4070,99.4
2,パチスロ ゼクスイグニッション,3,-1670,1990,72.0
3,麻雀格闘倶楽部２,5,-320,2370,95.5
...,...,...,...,...,...
307,プレミアムハナハナ-30,707,2810,7380,112.7
308,プレミアムハナハナ-30,708,-70,7220,99.7
309,プレミアムハナハナ-30,710,4160,9010,115.4
310,プレミアムハナハナ-30,711,4340,8850,116.3


<ipython-input-12-a53f97db94ce>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kisyubetu_sheet_df['勝利台数'] = kisyu_name_list
<ipython-input-12-a53f97db94ce>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kisyubetu_sheet_df['勝利台数'] = kisyubetu_sheet_df['勝利台数'].astype(str)
<ipython-input-12-a53f97db94ce>:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

,機種,差枚,G数,勝率,出率
110,Ｒｅ：ゼロから始める異世界生活,3110,6893,2/3,112.5
33,パチスロ モンキーターンIV,2363,9360,3/3,108.4
63,パチスロひぐらしのなく頃に祭２,1935,8282,6/6,107.8
105,麻雀物語３役満乱舞の究極大戦,1795,3620,2/2,114.1
...,...,...,...,...,...
92,押忍！番長３,-1358,2982,0/8,82.4
81,リノ,-1486,1224,1/5,58.9
93,政宗２,-1667,3320,0/3,79.6
113,ＳＬＯＴギルティクラウン,-4275,4705,0/2,69.8


In [1]:
#③実行する
import gspread
import json
#ServiceAccountCredentials：Googleの各サービスへアクセスできるservice変数を生成します。
from oauth2client.service_account import ServiceAccountCredentials 

#2つのAPIを記述しないとリフレッシュトークンを3600秒毎に発行し続けなければならない
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']

#認証情報設定
#ダウンロードしたjsonファイル名をクレデンシャル変数に設定（秘密鍵、Pythonファイルから読み込みしやすい位置に置く）
credentials = ServiceAccountCredentials.from_json_keyfile_name( r"C:\Users\81807\Downloads\記事作成用\twitteranalytics-jsonsercretkey.json", scope)
#r'C:\Users\4-22.TATSUMI\Desktop\記事作成用\twitteranalytics-jsonsercretkey.json'

#OAuth2の資格情報を使用してGoogle APIにログインします。
gc = gspread.authorize(credentials)

workbook = gc.open_by_url('https://docs.google.com/spreadsheets/d/1LmkbYAJq2g684bjF-vumexLgKrFQNsF-dhsdTv33_Yk/edit#gid=0')
workbook.add_worksheet(title=title_text_only, rows=700, cols=6)
worksheet = workbook.worksheet(title_text_only)


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\81807\\Downloads\\記事作成用\\twitteranalytics-jsonsercretkey.json'

In [15]:


#df_a['G数'] = df_a['G数'].map(set_round)
sum_append_list = []
df_a['差枚'] = pd.to_numeric(df_a['差枚'], downcast='integer')
df_a['G数'] = pd.to_numeric(df_a['G数'], downcast='integer')
df_a = df_a.reset_index(drop=True)
for i in range(0,len(df_a)):
    #print(df_a[i:i+6].sum()['差枚'])
    sum_append_list.append(df_a[i:i+6].sum()['差枚'])
df_a['6行合計値'] = sum_append_list
#display(df_a.style.bar(align="mid",color=['#d65f5f', '#5fba7d']))
df_a = df_a.drop('6行合計値',axis=1)
#ここで出ている箇所を確認する


In [ ]:
# 数字からアルファベットを返す関数
# 例：26→Z、27→AA、10000→NTP
def toAlpha(num):
    if num<=26:
        return chr(64+num)
    elif num%26==0:
        return toAlpha(num//26-1)+chr(90)
    else:
        return toAlpha(num//26)+chr(64+num%26)

col_lastnum = len(df_a.columns) # DataFrameの列数
row_lastnum = len(df_a.index)   # DataFrameの数

cell_list = worksheet.range('A1:'+toAlpha(col_lastnum)+str(row_lastnum))
for cell in cell_list:
    val = df_a.iloc[cell.row-1][cell.col-1]
    cell.value = str(val)
    print(cell.value)
worksheet.update_cells(cell_list)

In [17]:
#選択　並びピックアップ用

pd.set_option('display.max_rows', 8)
worksheet_df = pd.DataFrame(worksheet.get_all_values())
worksheet_df = worksheet_df.rename(columns={0: '機種', 1: '台番', 2: '差枚', 3: 'G数', 4: '出率', 5: '対象'})
#display(worksheet_df)

#ここに6台並びと思われる最初の台番号をいれる
pickup_df_text = ''
u = worksheet_df['対象'].unique()
for target_num in worksheet_df['対象'].unique():
    if target_num == '':
        print('continue')
        continue
    df_b = worksheet_df.query('対象.str.contains(@target_num)', engine='python')
    df_b = df_b.drop("対象", axis=1)
    display(df_b)
    print(df_b.to_html(justify='justify-all',index=False))

    pickup_df_text += df_b.to_html(justify='justify-all',index=False)
pickup_text = '<h2>注目ピックアップ</h2>\n[st-kaiwa1]今回の並びでよかった場所がこちら[/st-kaiwa1]'
pickup_text_2 = '\n[st-kaiwa1]今回の並びの一言コメントを記入[/st-kaiwa1]'
#print(narabi_kisyu_df_text)

continue


,機種,台番,差枚,G数,出率
43,パチスロ 北斗の拳 新伝説創造,65,4540,7860,119.3
123,Ｒｅ：ゼロから始める異世界生活,185,4280,7570,118.8
124,Ｒｅ：ゼロから始める異世界生活,186,6260,7860,126.5
169,パチスロ ガメラ,262,2130,6120,111.6
...,...,...,...,...,...
187,ハナビ,285,1410,7670,106.1
206,新ハナビ,318,1090,8160,104.5
288,シャア専用パチスロ 逆襲の赤い彗星,673,4070,9280,114.6
293,パチスロ頭文字Ｄ,680,580,8950,102.2


<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: justify-all;">
      <th>機種</th>
      <th>台番</th>
      <th>差枚</th>
      <th>G数</th>
      <th>出率</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>パチスロ　北斗の拳　新伝説創造</td>
      <td>65</td>
      <td>4540</td>
      <td>7860</td>
      <td>119.3</td>
    </tr>
    <tr>
      <td>Ｒｅ：ゼロから始める異世界生活</td>
      <td>185</td>
      <td>4280</td>
      <td>7570</td>
      <td>118.8</td>
    </tr>
    <tr>
      <td>Ｒｅ：ゼロから始める異世界生活</td>
      <td>186</td>
      <td>6260</td>
      <td>7860</td>
      <td>126.5</td>
    </tr>
    <tr>
      <td>パチスロ　ガメラ</td>
      <td>262</td>
      <td>2130</td>
      <td>6120</td>
      <td>111.6</td>
    </tr>
    <tr>
      <td>エヴァンゲリヲン　魂を繋ぐもの</td>
      <td>267</td>
      <td>1070</td>
      <td>6140</td>
      <td>105.8</td>
    </tr>
    <tr>
      <td>ハナビ通</td>
      <td>273</td>
      <td>3320</td>
      <td>9160</td>
      <td>112.1</td>
    </tr>
    <tr>
      <td>バーサス</td>


,機種,台番,差枚,G数,出率
46,パチスロひぐらしのなく頃に祭２,68,3520,8630,113.6
47,パチスロひぐらしのなく頃に祭２,70,2310,8190,109.4
48,パチスロひぐらしのなく頃に祭２,71,1490,7760,106.4
49,パチスロひぐらしのなく頃に祭２,72,1820,8540,107.1
50,パチスロひぐらしのなく頃に祭２,73,1710,8030,107.1
51,パチスロひぐらしのなく頃に祭２,75,760,8540,103.0


<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: justify-all;">
      <th>機種</th>
      <th>台番</th>
      <th>差枚</th>
      <th>G数</th>
      <th>出率</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>パチスロひぐらしのなく頃に祭２</td>
      <td>68</td>
      <td>3520</td>
      <td>8630</td>
      <td>113.6</td>
    </tr>
    <tr>
      <td>パチスロひぐらしのなく頃に祭２</td>
      <td>70</td>
      <td>2310</td>
      <td>8190</td>
      <td>109.4</td>
    </tr>
    <tr>
      <td>パチスロひぐらしのなく頃に祭２</td>
      <td>71</td>
      <td>1490</td>
      <td>7760</td>
      <td>106.4</td>
    </tr>
    <tr>
      <td>パチスロひぐらしのなく頃に祭２</td>
      <td>72</td>
      <td>1820</td>
      <td>8540</td>
      <td>107.1</td>
    </tr>
    <tr>
      <td>パチスロひぐらしのなく頃に祭２</td>
      <td>73</td>
      <td>1710</td>
      <td>8030</td>
      <td>107.1</td>
    </tr>
    <tr>
      <td>パチスロひぐらしのなく頃に祭２</td>
      <td>75</td>
      <td>760</td>
      <td>8540</td>
      <td>103.0</td>
    </tr>
  </tbody>
</tabl

...

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: justify-all;">
      <th>機種</th>
      <th>台番</th>
      <th>差枚</th>
      <th>G数</th>
      <th>出率</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>スーパーミラクルジャグラー</td>
      <td>371</td>
      <td>970</td>
      <td>5060</td>
      <td>106.4</td>
    </tr>
    <tr>
      <td>スーパーミラクルジャグラー</td>
      <td>372</td>
      <td>-620</td>
      <td>5100</td>
      <td>95.9</td>
    </tr>
    <tr>
      <td>スーパーミラクルジャグラー</td>
      <td>373</td>
      <td>1210</td>
      <td>3990</td>
      <td>110.1</td>
    </tr>
    <tr>
      <td>スーパーミラクルジャグラー</td>
      <td>375</td>
      <td>3500</td>
      <td>5900</td>
      <td>119.8</td>
    </tr>
    <tr>
      <td>スーパーミラクルジャグラー</td>
      <td>376</td>
      <td>1900</td>
      <td>7260</td>
      <td>108.7</td>
    </tr>
    <tr>
      <td>スーパーミラクルジャグラー</td>
      <td>377</td>
      <td>-1100</td>
      <td>2370</td>
      <td>84.5</td>
    </tr>
  </tbody>
</table>


,機種,台番,差枚,G数,出率
246,ファンキージャグラー,378,3390,8350,113.5
250,ファンキージャグラー,383,1690,7790,107.2
266,マイジャグラーＩＩＩ,513,270,8930,101.0
275,マイジャグラーＩＶ,525,2550,8760,109.7


<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: justify-all;">
      <th>機種</th>
      <th>台番</th>
      <th>差枚</th>
      <th>G数</th>
      <th>出率</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>ファンキージャグラー</td>
      <td>378</td>
      <td>3390</td>
      <td>8350</td>
      <td>113.5</td>
    </tr>
    <tr>
      <td>ファンキージャグラー</td>
      <td>383</td>
      <td>1690</td>
      <td>7790</td>
      <td>107.2</td>
    </tr>
    <tr>
      <td>マイジャグラーＩＩＩ</td>
      <td>513</td>
      <td>270</td>
      <td>8930</td>
      <td>101.0</td>
    </tr>
    <tr>
      <td>マイジャグラーＩＶ</td>
      <td>525</td>
      <td>2550</td>
      <td>8760</td>
      <td>109.7</td>
    </tr>
  </tbody>
</table>


,機種,台番,差枚,G数,出率
299,プレミアムハナハナ-30,687,2850,6880,113.8
300,プレミアムハナハナ-30,688,1860,5230,111.9
301,プレミアムハナハナ-30,700,-500,4140,96.0
302,プレミアムハナハナ-30,701,180,6380,100.9
...,...,...,...,...,...
307,プレミアムハナハナ-30,707,2810,7380,112.7
308,プレミアムハナハナ-30,708,-70,7220,99.7
309,プレミアムハナハナ-30,710,4160,9010,115.4
310,プレミアムハナハナ-30,711,4340,8850,116.3


<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: justify-all;">
      <th>機種</th>
      <th>台番</th>
      <th>差枚</th>
      <th>G数</th>
      <th>出率</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>プレミアムハナハナ-30</td>
      <td>687</td>
      <td>2850</td>
      <td>6880</td>
      <td>113.8</td>
    </tr>
    <tr>
      <td>プレミアムハナハナ-30</td>
      <td>688</td>
      <td>1860</td>
      <td>5230</td>
      <td>111.9</td>
    </tr>
    <tr>
      <td>プレミアムハナハナ-30</td>
      <td>700</td>
      <td>-500</td>
      <td>4140</td>
      <td>96.0</td>
    </tr>
    <tr>
      <td>プレミアムハナハナ-30</td>
      <td>701</td>
      <td>180</td>
      <td>6380</td>
      <td>100.9</td>
    </tr>
    <tr>
      <td>プレミアムハナハナ-30</td>
      <td>702</td>
      <td>4590</td>
      <td>5750</td>
      <td>126.6</td>
    </tr>
    <tr>
      <td>プレミアムハナハナ-30</td>
      <td>703</td>
      <td>-160</td>
      <td>7750</td>
      <td>99.3</td>
    </tr>
    <tr>
      <td>プレミアムハナハナ-30

In [9]:
#選択② 並び取材ならここはパスする
#単品ピックアップ用
df_a.dtypes
df_a['台番'] = pd.to_numeric(df_a['台番'], downcast='integer')
df_pickup = df_a[df_a['差枚'] > 1000]
df_pickup = df_pickup[df_pickup['G数'] > 6500]
df_pickup = df_pickup.sort_values('台番')
pickup_df_text = df_pickup.to_html(justify='justify-all',index=False)
pickup_text = '<h2>注目ピックアップ</h2>\n[st-kaiwa1]差枚+1000枚以上&回転数6500G以上の単品をピックアップしました[/st-kaiwa1]'
pickup_text_2 = '''\n[st-kaiwa1]コメント1行目
コメント2行目[/st-kaiwa1]'''
print(pickup_df_text)

NameError: name 'df_a' is not defined

In [18]:
#html出力用

title_text_only.split('_')[1]
title_text_only.split('_')[0]
from datetime import datetime as dt

tstr = title_text_only.split('_')[0]
tdatetime = dt.strptime(tstr, '%Y%m%d')
month = tdatetime.strftime('%m').lstrip('0')
day = tdatetime.strftime('%d').lstrip('0')
date_str = month + '/' + day 
date_str

import requests
from bs4 import BeautifulSoup

#タイトル部分の出力

title_text_only
tenpo_name = title_text_only.split('_')[1]
day_number = title_text_only.split('_')[0][-1]
title = '<h2>' + date_str + ' ' + tenpo_name + '</h2>'


day_number = day_number.replace('(月','').replace('(火','').replace('(水','').replace('(木','').replace('(金','').replace('(土','').replace('(日','')

day_number

#print(title)

tenpo_ave_gamesuu = int(sheet_df['G数'].sum() / len(sheet_df['G数']))
tenpo_ave_gamesuu = str("{:,}".format(tenpo_ave_gamesuu)) + 'G'
tenpo_ave_samai = str(int(sheet_df['差枚'].sum() / len(sheet_df['G数']))) + '枚'
tenpo_sousamai = sheet_df['差枚'].sum()
tenpo_sousamai = str("{:,}".format(tenpo_sousamai )) + '枚'

win_rate_bunshi = 0
for x in list(sheet_df['差枚']):
    if x > 0:
        win_rate_bunshi += 1
    else:
        pass
win_rate = str(win_rate_bunshi)+'/'+str(len(sheet_df['G数']))

#概要部分の出力
day_number = title_text_only.split('_')[0][-1]
tenpo_gaiyou_dict = {'状況': ['総差枚', '平均差枚', '平均G数', '勝率'], f'旧イベント日（{day_number}のつく日）':[tenpo_sousamai,tenpo_ave_samai, tenpo_ave_gamesuu, win_rate]}

matome_df = pd.DataFrame.from_dict(tenpo_gaiyou_dict)

kisyu_matome_df = kisyubetu_sheet_df



gaiyou_matome = matome_df.to_html(justify='justify-all',index=False)
#print(gaiyou_matome)

#機種の全台データ一覧
#print(ichiran_df.to_html(justify='justify-all',index=False))
text = ichiran_df.to_html(justify='justify-all',index=False)
completed_text ='[su_spoiler title="全台データ一覧" style="fancy" icon="chevron-circle" anchor="Hello"]' + text + '[/su_spoiler]'

#差枚G数グラフ一覧
#kisyu_matome_df = pd.read_html(url)[1]

horaizontal_bar_graph = """<script src="https://cdnjs.cloudflare.com/ajax/libs/Chart.js/2.1.4/Chart.min.js"></script>
<canvas id="myChart" height="1500" style="padding: 0px 10px 0px 10px;"></canvas>
<script>
var ctx = document.getElementById("myChart").getContext('2d');
var myChart = new Chart(ctx, {
  type: 'horizontalBar',
  data:  {
    labels:""" + str(kisyu_matome_df['機種'].to_list()) + """,
    datasets: [{
        label: "平均差枚",
        data:""" + str(kisyu_matome_df['差枚'].to_list()) +""",
        borderColor: "rgb(255, 99, 132)",
        backgroundColor: "red"
      },{
        label: "平均G数",
        data:""" + str(kisyu_matome_df['G数'].to_list()) + """,
backgroundColor: "blue"
    }
]
  }
});
</script>"""
#print(horaizontal_bar_graph)

kisyu_matome_text = ''
#機種別テーブル
#kisyu_matome_df = pd.read_html(url)[1]

for kisyu_name in kisyu_matome_df['機種']:
    target_matome_kisyu = kisyu_matome_df.query(f'機種.str.contains("{kisyu_name}")', engine='python')
    try:
        kisyu_text = f'{str(target_matome_kisyu.iloc[0,0])}<br> 平均差枚:{str(target_matome_kisyu.iloc[0,1])}枚<br> 平均G数:{str(target_matome_kisyu.iloc[0,2])}G<br>  勝率:{str(target_matome_kisyu.iloc[0,3])}<br>  出率:{str(target_matome_kisyu.iloc[0,4])}%'
    except:
        continue
    kisyu_matome_text += '[su_spoiler title="' + kisyu_text + '" style="fancy" icon="chevron-circle" anchor="Hello"]'
    #print('[su_spoiler title="',kisyu_text,'" style="fancy" icon="chevron-circle"]')
    
    target_kisyu = ichiran_df.query(f'機種.str.contains("{kisyu_name}")', engine='python')
    target_kisyu['出率'] = target_kisyu['出率'].astype(str) + '%'
    target_kisyu['差枚'] = target_kisyu['差枚'].astype(str) + '枚'
    target_kisyu['G数'] = target_kisyu['G数'].astype(str) + 'G'
    kisyu_matome_text += target_kisyu.to_html(justify='justify-all',index=False) + '\n'
    #print(target_kisyu.to_html(justify='justify-all',index=False))
    
    kisyu_matome_text += '[/su_spoiler]' + '\n'
    #print('[/su_spoiler]')


zizen_text = '\n<h2>事前情報</h2>\n画像を入れてください\n[st-kaiwa1]一言コメント部分[/st-kaiwa1]'

full_text  = (title + '\n' + gaiyou_matome + zizen_text + '<h2>機種別平均差枚&平均G数グラフ</h2>'+ horaizontal_bar_graph +  pickup_text + pickup_df_text + pickup_text_2 + "<h2>全台データ一覧</h2>※タップで一覧が開きます"+ completed_text + '<h2>機種別データ一覧</h2>※機種タップで各台データが見れます' + kisyu_matome_text)
print('準備完了')

960d2>:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_kisyu['G数'] = target_kisyu['G数'].astype(str) + 'G'
<ipython-input-18-10bb92d960d2>:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_kisyu['出率'] = target_kisyu['出率'].astype(str) + '%'
<ipython-input-18-10bb92d960d2>:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

In [19]:
# -*- coding: utf-8 -*-
import time
from datetime import datetime
from wordpress_xmlrpc import Client, WordPressPost
from wordpress_xmlrpc.methods.users import GetUserInfo
from wordpress_xmlrpc.methods.posts import GetPosts, NewPost
def main():
    """
    変数を定義
    """
    id = "oshihall"
    password="slopachi777"
    #idとpasswordはwordpressの管理画面に入るためのもの

    url="https://slotoshihall.com/xmlrpc.php"
    #第3者が閲覧するURLの後ろに/xmlrpc.phpをつける。
    #ワードプレスの管理画面の後ろにつけるとエラーになった

    which="draft"
    #which="draft"
    #下書きに投稿するか本番で投稿するか選択
    """
    クライアントの呼び出しなど
    """

    wp = Client(url, id,password)
    post = WordPressPost()

    """
    実際に投稿する
    """

    post.post_status = which
    post.title = date_str +' '+tenpo_name + '結果まとめ'
    post.content = full_text
    post.terms_names = {
    "post_tag": [f'{day_number}のつく日',tenpo_name],
    "category": ['結果記事'],
    }
    #過去に投稿した記事としたい場合、投稿日をここで指定。例として2018年1月1日10時5分10秒に投稿した例を示す。
    #post.date=datetime.strptime("2018/1/01 10:05:10","%Y/%m/%d %H:%M:%S")
    wp.call(NewPost(post))

main()
#ここまでで下書きに記事完成

In [10]:
"""WORDPRESSの操作"""
import json
import os
import base64
import requests


class WordPressError(Exception):
    """WordPressのエラー情報"""
    def __init__(self, ctrl, status_code, reason, message):
        super(WordPressError, self).__init__()
        self.ctrl = ctrl
        self.status_code = status_code
        self.reason = reason
        self.message = message


class WordPressCtrl:
    """WordPressの操作"""

    def __init__(self, url, user, password):
        """初期化処理"""
        self.url = url
        auth_str = f"{user}:{password}"
        auth_base64_bytes = base64.b64encode(auth_str.encode(encoding='utf-8'))
        self.auth = auth_base64_bytes.decode(encoding='utf-8')

    def check_response(self, res, success_code):
        """WordPressからの応答をチェック"""
        try:
            json_object = json.loads(res.content)
        except ValueError as ex:
            raise WordPressError(self, res.status_code, res.reason, str(ex))
        if res.status_code != success_code:
            raise WordPressError(self, res.status_code, res.reason, json_object['message'])
        return json_object

    def add_post(self, title, content, categorie_ids=[], tag_ids=[]):
        """WordPressに記事を投稿"""
        headers = {
            'Authorization': 'Basic ' + self.auth
        }
        data = {
            'title': title,
            'content': content,
            'format': 'standard',
            'categories' : categorie_ids,
            'tags' : tag_ids
        }
        res = requests.post(f'{self.url}/wp-json/wp/v2/posts', json=data, headers=headers)
        return self.check_response(res, 201)

    def update_post(self, id, title, content, categorie_ids=[], tag_ids=[]):
        """WordPressの既存記事を更新"""
        headers = {
            'Authorization': 'Basic ' + self.auth
        }
        data = {
            'title': title,
            'content': content,
            'format': 'standard',
            'categories' : categorie_ids,
            'tags' : tag_ids
        }
        res = requests.post(f'{self.url}/wp-json/wp/v2/posts/{id}', json=data, headers=headers)
        return self.check_response(res, 200)

    def upload_media(self, path, content_type):
        """メディアのアップロード"""
        file_name = os.path.basename(path)
        headers = {
            'Authorization': 'Basic ' + self.auth,
            'Content-Type': content_type,
            'Content-Disposition' : 'attachiment; filename={filename}'.format(filename=file_name)
        }
        with open(path, 'rb') as media_file:
            data = media_file.read()
        res = requests.post(f'{self.url}/wp-json/wp/v2/media', data=data, headers=headers)
        return self.check_response(res, 201)

    def upload_png(self, path):
        """メディアにPNG画像を追加"""
        return self.upload_media(path, 'image/png')

    def upload_jpeg(self, path):
        """メディアにJPEG画像を追加"""
        return self.upload_media(path, 'image/jpeg')


In [20]:
def set_round(x):
    x = int(round(x, 1))
    return x

df_a['G数'] = df_a['G数'].map(set_round)
df_a.style.bar(align="mid",color=['#d65f5f', '#5fba7d'])
n=0
sum_append_list = []
for i in range(0,len(df_a)):
    print(df_a[i:i+6].sum()['差'])
    sum_append_list.append(df_a[i:i+6].sum()['差'])
df_a['6行合計値'] = sum_append_list


df_a['台番号'] = df_a['台番号'].astype(str)
for number in ['610','682','1057','1101']:
    df_b = df_a.query('台番号.str.contains(@number)', engine='python')
    #display(df_b)
    index_number = df_b.index[0]
    display(df_a[index_number:index_number+6])
    print(index_number)

KeyError: '差'